# Import data

In [1]:
import pandas as pd
import os
import glob

In [2]:
## HESITEZ PAS A RENDRE CETTE FONCTION PLUS MODULAIRE

# Get only columns we need (for now county_name and search_conducted)
def filterCols(df):
    if ('county_name' in df.columns) & ('search_conducted' in df.columns):
        if 'contraband_found' in df.columns:
            return df[['date','county_name', 'subject_race', 'officer_race', 'search_conducted', 'contraband_found']]
        else:
            return df[['date','county_name', 'subject_race', 'officer_race', 'search_conducted']]
    else :
            return df[['date', 'subject_race', 'officer_race']]

In [3]:
FOLDER = 'raw/'
FOLDER_JAMES = 'data/data_officer'

csvfiles = glob.glob(os.path.join(FOLDER_JAMES, '*.csv'))

data = pd.DataFrame(columns=['date', 'time', 'state', 'county_name', 'subject_race', 'officer_race', 'search_conducted', 'contraband_found'])

chunksize = 10 ** 6
for csvfile in csvfiles:
    for chunk in pd.read_csv(csvfile, usecols=lambda x: x in ['date', 'time', 'county_name', 'subject_race', 'officer_race', 'search_conducted', 'contraband_found'] \
                     , chunksize=chunksize, low_memory=False, error_bad_lines = False):
        chunk['state'] = csvfile[18:20]
        if 'county_name' not in chunk.columns:
            end=0
            for i in range(len(csvfile)) :
                if csvfile[i].isdigit() :
                    end = i
                    break
            county_name = csvfile[21:end]
            county_name = county_name.replace('_', ' ')
            chunk['county_name']=county_name
        data = pd.concat([data, chunk], axis=0 , ignore_index=True, sort=False)

In [4]:
data.head(10)

,date,time,state,county_name,subject_race,officer_race,search_conducted,contraband_found
0,2008-10-10,NaN,ca,long beach,hispanic,hispanic,NaN,NaN
1,2008-10-10,NaN,ca,long beach,hispanic,white,NaN,NaN
2,2008-10-10,NaN,ca,long beach,hispanic,white,NaN,NaN
3,2008-10-10,NaN,ca,long beach,white,white,NaN,NaN
4,2008-10-10,NaN,ca,long beach,hispanic,white,NaN,NaN
5,2008-10-10,NaN,ca,long beach,hispanic,white,NaN,NaN
6,2008-10-10,NaN,ca,long beach,black,white,NaN,NaN
7,2008-10-10,NaN,ca,long beach,black,hispanic,NaN,NaN
8,2008-10-10,NaN,ca,long beach,black,white,NaN,NaN
9,2008-10-10,NaN,ca,long beach,black,hispanic,NaN,NaN


## Clean data

In [5]:
# Convert date to datetime
data['date'] = pd.to_datetime(data['date'], infer_datetime_format=True).dt.date

In [6]:
# Remove fields where race unknown
data = data[(data['subject_race'] != 'unknown') & (data['officer_race'] != 'unknown')]

## DO NOT REMOVE NAN VALUES HERE SINCE SOME DATA SETS WERE MISSING SOME INFOS
## (EX COUNTY_NAME) BUT HAVE OTHER USEFUL FIELDS

FILTER ACCORDING TO TIME TOO

In [7]:
data.to_csv(r'data_clean/data.csv', index=False)